In [1]:
# 서울시 교통사고 시각화

In [2]:
import pandas as pd

In [3]:
traffic_Seoul = pd.read_csv("../Data/newSeoul_2005_2019.csv")
traffic_Seoul.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [4]:
# 2019년 데이터만 추출하기
# traffic_Seoul.info() -> 년도 int64
traffic_Seoul_2019 = traffic_Seoul[traffic_Seoul['년도'] == 2019]
# traffic_Seoul_2019['년도'].unique()

# index 값 0부터 시작하게 바꾸기
traffic_Seoul_2019.reset_index(drop=True, inplace=True)
traffic_Seoul_2019.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158


In [5]:
# 자치구별, 발생건수, 사망자수, 부상자수, 집계하기
traffic_analysis = pd.pivot_table(traffic_Seoul_2019[['자치구명','발생건수','부상자수','사망자수']], \
                                    index='자치구명' , aggfunc=sum)
traffic_analysis.head()


,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [6]:
# 위도, 경도 데이터 불러오기
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb0 in position 13: invalid start byte
# 윈도우에서 만든 데이터라서 에러 걸림 ->  encoding='euc-kr' 를 써야함.
seoul_limit = pd.read_csv("../Data/seoul.csv", encoding='euc-kr')
seoul_limit.columns = ['자치구명','lon','lat']
seoul_limit.head()

,자치구명,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [7]:
# traffic_Seoul_2019 와 seoul_limit의 병합
data_result =  pd.merge(traffic_analysis, seoul_limit, on='자치구명')
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,강남구,3722,5182,14,127.0475,37.51731
1,강동구,1414,1910,11,127.1238,37.53013
2,강북구,1277,1706,7,127.0255,37.63975
3,강서구,1829,2491,20,126.8496,37.55094
4,관악구,1363,1755,10,126.9515,37.47834


In [8]:
data_result.tail()

,자치구명,발생건수,부상자수,사망자수,lon,lat
20,용산구,1187,1614,10,126.9904,37.53254
21,은평구,1119,1464,7,126.9292,37.60278
22,종로구,1133,1559,6,126.9793,37.57323
23,중구,1184,1615,5,127.0032,37.55273
24,중랑구,1620,2122,12,127.0928,37.60652


In [9]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     int64  
 2   부상자수    25 non-null     int64  
 3   사망자수    25 non-null     int64  
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.4+ KB


In [10]:
# folium을 사용하기 위해 int를 float로 변환해야 한다.
data_result[['발생건수','부상자수','사망자수']]=data_result[['발생건수','부상자수','사망자수']].astype(float)
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.4+ KB


### 자치구의 구청을 표시하기

In [11]:
import folium

In [12]:
map = folium.Map(
    location=[37.5520, 126.982],
    zoom_start=11,
)

# 위치 정보를 Marker로 표시
for n in data_result.index:
    folium.Marker([data_result['lat'][n], data_result['lon'][n]],
    popup=data_result['자치구명'][n]).add_to(map)

map


In [13]:
map = folium.Map(
    location=[37.5520, 126.982],
    zoom_start=11,
)

# 위치 정보를 Marker로 표시
for n in data_result.index:
    folium.Marker([data_result['lat'][n], data_result['lon'][n]],
    popup=data_result['자치구명'][n]).add_to(map)
    

# 자치구별 교통사고 발생건수를 표시하기. (숫자 데이터)
for n in data_result.index:
    folium.CircleMarker(
                [data_result['lat'][n], data_result['lon'][n]],
                radius=data_result['발생건수'][n]/100,
                fill = True,
                fill_opacity=0.3,
                popup=data_result['발생건수'][n]).add_to(map)

map


In [14]:
# 자치구별 교통사고 사망건수 표시하기
map = folium.Map(
    location=[37.5520, 126.982],
    zoom_start=11,
)

# 위치 정보를 Marker로 표시
for n in data_result.index:
    folium.Marker([data_result['lat'][n], data_result['lon'][n]],
    popup=data_result['자치구명'][n]).add_to(map)
    

# 자치구별 교통사고 사망자수를 표시하기. (숫자 데이터)
for n in data_result.index:
    folium.CircleMarker(
                [data_result['lat'][n], data_result['lon'][n]],
                radius=data_result['사망자수'][n] * 2,
                fill = True,
                fill_opacity=0.3,
                color = 'firebrick',
                popup=data_result['사망자수'][n]).add_to(map)

map


In [15]:
# 사망자수 정렬후 index 재설정
data_result2 = data_result.sort_values(by='사망자수', ascending=False)
data_result2.reset_index(drop=True, inplace=True)
data_result2.tail()

,자치구명,발생건수,부상자수,사망자수,lon,lat
20,종로구,1133.0,1559.0,6.0,126.9793,37.57323
21,동작구,1406.0,2021.0,5.0,126.9393,37.51243
22,중구,1184.0,1615.0,5.0,127.0032,37.55273
23,도봉구,852.0,1163.0,4.0,127.0478,37.66873
24,광진구,973.0,1316.0,3.0,127.0824,37.53862


In [16]:
# 자치구별 교통사고 사망건수 표시하기
map = folium.Map(
    location=[37.5520, 126.982],
    zoom_start=11,
)

# 위치 정보를 Marker로 표시
for n in data_result2.index:
    folium.Marker([data_result2['lat'][n], data_result2['lon'][n]],
    popup=data_result2['자치구명'][n]).add_to(map)
    

# 자치구별 교통사고 사망자수의 최대값과 최소값을 표시하기. (숫자 데이터)
folium.CircleMarker(
            [data_result2['lat'][0], data_result2['lon'][0]],
            radius=data_result2['사망자수'][0] * 2,
            fill = True,
            fill_opacity=0.3,
            color = 'firebrick',
            popup=data_result2['사망자수'][0]).add_to(map)
folium.CircleMarker(
            [data_result2['lat'][len(data_result2)-1], data_result2['lon'][len(data_result2)-1]],
            radius=data_result2['사망자수'][len(data_result2)-1] * 2,
            fill = True,
            fill_opacity=0.3,
            color = 'indigo',
            popup=data_result2['사망자수'][len(data_result2)-1]).add_to(map)
map

In [17]:
# 위에(치구별 교통사고 사망자수의 최대값과 최소값을 표시하기)랑 같은 그래프 그리기 다른버전

map = folium.Map(
    location=[37.5502, 126.982],
    zoom_start=11
)
for name, lat, lon, dead in zip(data_result.자치구명, data_result.lat, data_result.lon, data_result.사망자수):

    folium.Marker([lat,lon], popup=name).add_to(map)

# 자치구별 교통사고 사망자수의 최대값과 최소값을 표시하기. (숫자 데이터)
folium.CircleMarker(
                        [data_result.loc[data_result.사망자수 == data_result.사망자수.max(), "lat"],
                         data_result.loc[data_result.사망자수 == data_result.사망자수.max(), "lon"]],
                        radius = data_result.사망자수.max() * 1.9,
                        color = 'purple',
                        fill_color= 'purple',
                        fill_opacity= 0.7,
                        popup=name).add_to(map)

folium.CircleMarker(
                        [data_result.loc[data_result.사망자수 == data_result.사망자수.min(), "lat"],
                         data_result.loc[data_result.사망자수 == data_result.사망자수.min(), "lon"]],
                        radius = data_result.사망자수.min() * 1.9,
                        color = 'purple',
                        fill_color= 'purple',
                        fill_opacity= 0.7,
                        popup=name).add_to(map)

map